<h1>Section 1: Graph-Tool<h1>
        This section includes testing of the basic Graph-Tool functionalities

In [ ]:
from graph_tool.all import *

In [2]:
g = Graph(directed=False)
v1 = g.add_vertex()
v2 = g.add_vertex()
v3 = g.add_vertex()
e1 = g.add_edge(v1, v2)
e2 = g.add_edge(v1, v3)

In [3]:
graph_draw(g, vertex_text=g.vertex_index, output="two-nodes.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1a178fb50, at 0x1a1799580>

In [4]:
print(v1.out_degree())
print(e1.source(), e1.target())

2
0 1


In [5]:
vlist = g.add_vertex(10)
print(len(list(vlist)))

10


In [6]:
e3 = g.add_edge(4, 5)

In [7]:
for v in g.iter_vertices():
    print(v)
for e in g.iter_edges():
    print(e)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
[0, 1]
[0, 2]
[4, 5]


In [8]:
g.get_out_degrees(g.get_vertices())

array([2, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint64)

In [17]:
from numpy.random import randint

g = Graph()
g.add_vertex(100)

# insert some random links
num_edges = 10
for s,t in zip(randint(0, 100, num_edges), randint(0, 100, num_edges)):
    g.add_edge(g.vertex(s), g.vertex(t))
    
#print edges
print("edges added:")
for i in g.iter_edges():
    print(i)

vprop_double = g.new_vertex_property("double")            # Double-precision floating point
v = g.vertex(10)
vprop_double[v] = 3.1416
print("property of vertex {}: {}".format(v, vprop_double[v]))

vprop_vint = g.new_vertex_property("vector<int>")         # Vector of ints
v = g.vertex(40)
vprop_vint[v] = [1, 3, 42, 54]
print("property of vertex {}: {}".format(v, vprop_vint[v]))

eprop_dict = g.new_edge_property("object")                # Arbitrary Python object.
e = g.edges().next()
eprop_dict[e] = {"name": "foo", "weight": 42} 
print("property of edge {}: {}".format(e, eprop_dict[e]))

gprop_bool = g.new_graph_property("bool")                 # Boolean
gprop_bool[g] = True


edges added:
[0, 94]
[3, 90]
[24, 37]
[29, 12]
[32, 25]
[40, 5]
[64, 2]
[65, 53]
[70, 61]
[71, 52]
property of vertex 10: 3.1416
property of vertex 40: array([ 1,  3, 42, 54], dtype=int32)
property of edge (0, 94): {'name': 'foo', 'weight': 42}


In [26]:
# See what the initializations are for each of the graph properties
for prop in ["int", "vector<int>", "double", "vector<double>", "object", "bool"]:
    gprop = g.new_graph_property(prop)
    g.graph_properties["custom property"] = gprop
    print(prop, g.graph_properties["custom property"])

int 0
vector<int> array([], dtype=int32)
double 0.0
vector<double> array([], dtype=float64)
object None
bool 0


In [30]:
g.vp.custom_property = vprop = g.new_vertex_property("bool")
g.vp.custom_property[g.vertex(0)] = True
print(g.vp.custom_property[g.vertex(0)])

1


In [ ]:
import os
working_dir = '.' #placeholder

'''
Saves the graph g to {target path} with name {name}
'''
def save_graph(g, name, target_path):
    abs_path = os.path.join(working_dir, target_path)
    g.save("{}.xml.gz".format(name))
    

'''
Loads the graph g from {source path} 

Returns: the loaded graph 
'''
def load_graph(source_path):
    return load_graph(source_path)


In [31]:
'''
converts the following to a unified graph:
    
    - agent ids
    - agent intrinsics
    - driving forces
    - opinion
    
Input:

    agents:          numpy array of agent intrinsics
    clusters:        {"cluster_i": a list of agent ids}
    driving_forces:  numpy array of the driving forces
    opinions:        binary numpy array of the opinions
    
'''
def to_graph(agents, clusters, driving_forces, opinions):
    pass
    

<h2>Section 2: the model will be implemented here<h2>

In [72]:
import numpy as np
import scipy
from scipy.stats import norm
import math

In [101]:
'''
Global configurations
'''
k_alpha = 1
k_beta = .5
E_profit = 10
K_self_coupling = -2

num_clusters = 5
num_agents = 20

In [112]:
from random import randint

'''
Create {num_clusters} random clusters
if cluster_specific = True, use initialise_driving_force() to initialise cluster-specific driving force

Returns: 
           cluster_elements:  {"cluster_i": numpy array of agent ids in cluster i}
           driving_forces:    numpy array of driving forces (properly-/0-initialised)
           agents:            numpy array of agent intrinsics
           opinions:          a numpy binary array of the opinions

'''

def create_clusters(num_clusters, num_agents, cluster_specific=False):
    assert num_agents >= num_clusters >= 1
    
    #contains the slices between clusters
    # TODO: needs better random partition
    clusters = []
    for idx in range(num_clusters-1):
        clusters.append(randint(1,num_agents-sum(clusters)-num_clusters+idx))
    clusters.append(num_agents-1)

    # create cluster elements
    agents_ids = np.arange(num_agents)
    np.random.shuffle(agents_ids)
    cluster_elements = {}
    curr_pos = 0
    for i in range(num_clusters):
        cluster_elements["cluster_{}".format(i)] = agents_ids[curr_pos:(clusters[i]+curr_pos-1)]
        curr_pos += clusters[i]
        
    #initialise driving forces
    if cluster_specific:
        driving_forces = initialise_driving_forces(num_agents)
    else:
        driving_forces = np.zeros(num_agents)
        
    #initialise agents intrinsics
    agents = initialise_agents(num_agents)
    
    #initialise opinions
    opinions = initialise_opinions(num_agents)
        
    return cluster_elements, driving_forces, agents, opinions
        
    
'''
Initialises the driving forces for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the driving force of each agent

'''   
def initialise_driving_forces(num_agents):
    driving_forces = np.random.rand(num_agents) #hard-coded for now
    return driving_forces
    
'''
Initialises the intrinsics for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the intrinsics of each agent
'''
def initialise_agents(num_agents):
    return np.zeros(num_agents) #hard-coded for now


'''
Initialises the opinions for each agent in {agents}
You can define initialization schemes here

Retuns: a numpy array containing the opinions of each agent
'''
def initialise_opinions(num_agents):
    return np.random.randint(2, num_agents) #hard-coded for now



In [113]:
clusters, driving_forces, agents, opinions = create_clusters(num_clusters, num_agents)
print("clusters: {}\n\n driving forces: {}\n\n agents:{}\n\n opinions:{}".format(clusters, driving_forces, agents, opinions))

clusters: {'cluster_0': array([ 4, 16,  5, 12,  6,  0, 19, 14, 10,  3,  8, 15, 18,  1]), 'cluster_1': array([], dtype=int64), 'cluster_2': array([], dtype=int64), 'cluster_3': array([], dtype=int64), 'cluster_4': array([13, 17])}

 driving forces: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

 agents:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



In [120]:
'''
    updates the agent intrinsics (optional), opinions and driving forces 
'''

# TODO: need to fix this
def update_driving_forces(driving_forces, agents, clusters, opinions):
    # update each cluster locally
    updated_agents = np.zeros(len(driving_forces))
    for n in range(len(clusters)):
        agents_ids = clusters["cluster_{}".format(n)]
        cluster_size = len(agents_ids)
        
        # update the driving force of each agent in the current cluster
        # based on the other agents in the cluster
        
#         for a in agents_ids:
#             updated_agents[a] = np.sum([x for x in driving_forces[agents_ids] if x != a])
            
#         for a in agents_ids:
#             driving_forces[a] = E_profit + np.sum(updated_agents[agents_ids]) * k_alpha
#             print("agent {} obatined a df of {}".format(a, driving_forces[a]))
#         print("resulting df: {}\n".format(driving_forces))
            
    return updated_agents, driving_forces

In [121]:
upd_df = update_driving_forces(driving_forces, opinions, clusters, opinions)
upd_df

agent 4 obatined a df of 8716782490.0
agent 16 obatined a df of 8716782490.0
agent 5 obatined a df of 8716782490.0
agent 12 obatined a df of 8716782490.0
agent 6 obatined a df of 8716782490.0
agent 0 obatined a df of 8716782490.0
agent 19 obatined a df of 8716782490.0
agent 14 obatined a df of 8716782490.0
agent 10 obatined a df of 8716782490.0
agent 3 obatined a df of 8716782490.0
agent 8 obatined a df of 8716782490.0
agent 15 obatined a df of 8716782490.0
agent 18 obatined a df of 8716782490.0
agent 1 obatined a df of 8716782490.0
resulting df: [8.71678249e+09 8.71678249e+09 0.00000000e+00 8.71678249e+09
 8.71678249e+09 8.71678249e+09 8.71678249e+09 0.00000000e+00
 8.71678249e+09 0.00000000e+00 8.71678249e+09 0.00000000e+00
 8.71678249e+09 7.00000000e+01 8.71678249e+09 8.71678249e+09
 8.71678249e+09 7.00000000e+01 8.71678249e+09 8.71678249e+09]

resulting df: [8.71678249e+09 8.71678249e+09 0.00000000e+00 8.71678249e+09
 8.71678249e+09 8.71678249e+09 8.71678249e+09 0.00000000e+00
 8.7

(array([6.2262732e+08, 6.2262732e+08, 0.0000000e+00, 6.2262732e+08,
        6.2262732e+08, 6.2262732e+08, 6.2262732e+08, 0.0000000e+00,
        6.2262732e+08, 0.0000000e+00, 6.2262732e+08, 0.0000000e+00,
        6.2262732e+08, 1.4000000e+02, 6.2262732e+08, 6.2262732e+08,
        6.2262732e+08, 1.4000000e+02, 6.2262732e+08, 6.2262732e+08]),
 array([8.71678249e+09, 8.71678249e+09, 0.00000000e+00, 8.71678249e+09,
        8.71678249e+09, 8.71678249e+09, 8.71678249e+09, 0.00000000e+00,
        8.71678249e+09, 0.00000000e+00, 8.71678249e+09, 0.00000000e+00,
        8.71678249e+09, 2.90000000e+02, 8.71678249e+09, 8.71678249e+09,
        8.71678249e+09, 2.90000000e+02, 8.71678249e+09, 8.71678249e+09]))

In [ ]:
'''
Given the current state, returns a new state

Input: 
       -current_state: (agents' instrinsics, opinions, probabilities of change, driving forces)
       -f:             the state transition function
       
Returns:
       The new state f()
'''
# TODO: adapt this 
# You can choose whether to unpack current_state or not
def update_state(current_state, f):
    return f(current_state)


'''
You can define state transition functions here
'''
def transition_function(state):
    pass

In [122]:
'''
Part of modelling the transition function

'''
def normalise_driving_force(driving_forces):
    hi = k_alpha*len(driving_forces) 
    lo = None

    driving_force_normed = driving_forces/hi
    probability = math.erf(driving_force_normed[0])
    print(driving_force_norm[0], probability)


'''
compute the probability of change given a driving force    
'''
def change_probs(driving_force, hi, lo=None):
    probs = np.zeros(len(driving_force))
    for i in range(len(driving_force)):
        if agents[i] < hi:
            probs[i] = math.erf(driving_force[i])#norm.ppf(agents[i])
        else:
            probs[i] = math.erf(hi)
    return probs

'''
The new opinion {x} of an agent given the prob. of change {p}

Can serve as a state transition function
'''
def opinion(x, p):
    res = (np.random.rand(1))[0]
    return 1-x if res < p else x

In [192]:
#update_state((updated_agents, probs), opinion)

[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]

<h1>Section 3: Simulation<h1>
    1.Preprocessing (e.g. what graph models to use)<br>
    2.Simulate the model and visualise the result (e.g. for each timestep)

In [ ]:
# Simulation configs
num_steps = 42
visualise_per = 5

# Simulation loop, e.g.
#
# model = preprocess(data)
#
# clusters, driving_forces, agents, opinions = create_clusters(num_clusters, num_agents)
# current_state = (clusters, driving_forces, agents, opinions)
#
# for i in range(num_steps):
#    if i % visualise_per = 0:
#       visualise(current_state)
#    current_state = update_state(current_state, transition_function)
#
# graph = to_graph(current_state)
# save_graph(graph, graph_name, save_path)
#
#
#
# Note that in the current version we need to convert the current state to a graph
# first in order to visualise it -> maybe improve this

    



